## Prince George's County (PGC) 311 Requests Analysis
1. Analyze the # of 311 requests for each type before and after lock down (March 2020)
2. Exploratay analysis on FEMS requests

### Some findings:
1. 39 types of service requests changed significantly after lock down (p-val <0.05)
2. t-test results can be found at the result directory (../result/PGC_service_type_ttest.csv) 
3. Box plots can be found at the figure directory (../figure/PGC_ServiceTypeBoxPlt/)

In [1]:
import numpy as np
import pandas as pd
import os
import re
import glob
import csv
import warnings
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import math
warnings.filterwarnings("ignore")
PGC_311_2019 = '/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/Prince George County/311_Prince George County_MD_2019.csv'
PGC_311_2020 = '/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/Prince George County/311_Prince George County_MD_2020.csv'

### All 311 Requests in PGC during 2019 to 2020

1. PGC 2019

In [2]:
PGC_311_2019_df = pd.read_csv(PGC_311_2019,index_col=0)
PGC_311_2019_df.head()

,Service Request #,Date Request Opened,Request Status,SLA,Service Group,Agency Responsible,Request Name,Street Address,Latitude,Longitude,Location,TNI Area Description,opened_date
0,19-00104025,12/19/2019 02:01:48 PM,CLOSED,CLOSED,Public Safety Communications,Homeland Security,PSC Audio Recording Request,NaN,NaN,NaN,NaN,County Address,2019-12-19 14:01:48
1,19-00104020,12/19/2019 01:33:38 PM,CLOSED,CLOSED,Traffic Operations Division,Department of Public Works & Transportation,Traffic Signs - Repair/Replace,3950 Garden City Dr,NaN,NaN,NaN,County Address,2019-12-19 13:33:38
2,19-00104015,12/19/2019 01:15:29 PM,CLOSED,CLOSED,Rva - General,Revenue Authority,Parking Complaints,511 ABEL AVE,38.883792,-76.920057,"(38.88379244, -76.92005729)",County Address,2019-12-19 13:15:29
3,19-00069381,07/30/2019 09:38:28 AM,CLOSED,CLOSED,Construction Division,Department of Public Works & Transportation,Roadway Emergencies,WOODYARD RD & DEBORAH ST,NaN,NaN,NaN,County Address,2019-07-30 09:38:28
4,19-00105069,12/27/2019 10:56:07 AM,CLOSED,CLOSED,Field Activities - Amd,Dept. of The Environment-DOE,Animal - Deceased,SUITLAND PKY & SOUTHERN AVE,NaN,NaN,NaN,County Address,2019-12-27 10:56:07


In [3]:
PGC_311_2019_df.columns

Index(['Service Request #', 'Date Request Opened', 'Request Status', 'SLA',
       'Service Group', 'Agency Responsible', 'Request Name', 'Street Address',
       'Latitude', 'Longitude', 'Location', 'TNI Area Description',
       'opened_date'],
      dtype='object')

In [4]:
PGC_311_2019_df['opened_date'] = pd.to_datetime(PGC_311_2019_df['Date Request Opened'], format='%m/%d/%Y %I:%M:%S %p')
PGC_311_2019_df = PGC_311_2019_df.loc[ PGC_311_2019_df['opened_date'].dt.year == 2019]
PGC_311_2019_df.drop_duplicates(subset=['Service Request #'], inplace = True)
min_date = min(PGC_311_2019_df['opened_date'])
max_date = max(PGC_311_2019_df['opened_date'])
print("#311 reports between " + str(min_date) + " and " + str(max_date) + ": " + str(PGC_311_2019_df.shape[0]))

#311 reports between 2019-01-01 01:21:16 and 2019-12-31 17:15:41: 57553


2. PGC 2020

In [5]:
PGC_311_2020_df = pd.read_csv(PGC_311_2020,index_col=0)
PGC_311_2020_df

,Service Request #,Date Request Opened,Request Status,SLA,Service Group,Agency Responsible,Request Name,Street Address,Latitude,Longitude,Location,TNI Area Description,opened_date
0,20-00067998,10/13/2020 07:40:26 AM,CLOSED,CLOSED,Police Department,Police Department,Traffic Hazard/Safety,4800 IVERSON PL,NaN,NaN,NaN,County Address,2020-10-13 07:40:26
1,20-00030537,05/30/2020 08:01:29 AM,CLOSED,CLOSED,Special Services Division,Department of Public Works & Transportation,Litter Removal (from County Right-of-Way),7167 ACCOKEEK RD,NaN,NaN,NaN,County Address,2020-05-30 08:01:29
2,20-00079887,12/01/2020 01:49:38 PM,CLOSED,CLOSED,Collections-WMD,Dept. of The Environment-DOE,Cancellation - Bulky Trash,3810 ASQUITH CT,NaN,NaN,NaN,County Address,2020-12-01 13:49:38
3,20-00079924,12/01/2020 03:04:16 PM,CLOSED,CLOSED,Special Services Division,Department of Public Works & Transportation,Tree Trimming in Public Right-of-Way,10604 CROOM RD,NaN,NaN,NaN,County Address,2020-12-01 15:04:16
4,20-00079861,12/01/2020 12:15:56 PM,CLOSED,CLOSED,Zoning,DPIE,ENF-Business Operation Concerns,4816 EMO ST,NaN,NaN,NaN,County Address,2020-12-01 12:15:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47140,20-00012020,02/24/2020 06:38:16 PM,OPEN,OVERDUE,Construction Division,Department of Public Works & Transportation,Sidewalk - Repair,10000 NICOL CT,38.922900,-76.833760,"(38.92289987, -76.83375981)",County Address,2020-02-24 18:38:16
47141,20-00012030,02/24/2020 09:44:09 PM,OPEN,OVERDUE,Construction Division,Department of Public Works & Transportation,Sidewalk - Repair,10106 ELGIN CIR,38.922476,-76.831500,"(38.92247581, -76.83149972)",County Address,2020-02-24 21:44:09
47142,20-00013370,03/02/2020 01:27:00 PM,OPEN,OPEN,Zoning,DPIE,ENF-Business/Commercial Exterior Concerns,111 WESTHAMPTON AVE,38.888102,-76.865453,"(38.88810163, -76.86545272)",County Address,2020-03-02 13:27:00
47143,20-00055745,08/27/2020 12:06:48 PM,OPEN,OPEN,Single Family (Code Enforcement),DPIE,ENF-Residential Property Concerns,16411 LEA DR,38.970168,-76.708779,"(38.97016759, -76.70877855)",County Address,2020-08-27 12:06:48


In [6]:
PGC_311_2020_df.columns

Index(['Service Request #', 'Date Request Opened', 'Request Status', 'SLA',
       'Service Group', 'Agency Responsible', 'Request Name', 'Street Address',
       'Latitude', 'Longitude', 'Location', 'TNI Area Description',
       'opened_date'],
      dtype='object')

In [7]:
PGC_311_2020_df['opened_date'] = pd.to_datetime(PGC_311_2020_df['Date Request Opened'], format='%m/%d/%Y %I:%M:%S %p')
PGC_311_2020_df = PGC_311_2020_df.loc[ PGC_311_2020_df['opened_date'].dt.year == 2020]
PGC_311_2020_df.drop_duplicates(subset=['Service Request #'], inplace = True)
min_date = min(PGC_311_2020_df['opened_date'])
max_date = max(PGC_311_2020_df['opened_date'])
print("#311 reports between " + str(min_date) + " and " + str(max_date) + ": " + str(PGC_311_2020_df.shape[0]))

#311 reports between 2020-01-01 08:08:42 and 2020-12-31 21:57:12: 47145


In [8]:
service_desc_2019 = PGC_311_2019_df['Request Name'].unique().tolist()
len(service_desc_2019)

177

In [9]:
service_desc_2020 = PGC_311_2020_df['Request Name'].unique().tolist()
len(service_desc_2020)

173

In [10]:
intersect_service_desc = set(service_desc_2019) & set(service_desc_2020)
print(len(intersect_service_desc))
intersect_service_desc

156


{'Air/Land/Water Contamination Complaint',
 'Alarm Registration (False Alarm Registration)',
 'Animal - At Large (Domestic)',
 'Animal - Barking Complaint',
 'Animal - Bite',
 'Animal - Cruelty',
 'Animal - Deceased',
 'Animal - Illegal Breeds',
 'Animal - Injured',
 'Animal - Lost & Found',
 'Animal - Noise (i.e. dogs/chickens)',
 'Animal - Not Customarily Household Pet',
 'Animal - Vicious',
 'Asphalt Curb Repair/Installation',
 'Boat Trailer/Camping Trailer/Equipment Trailer on Roadway',
 'Burial Assistance Program',
 'Bus Stop - New',
 'Bus Stop Trash Can Overflowing',
 'Cable Commission Complaint',
 'Call-a-Bus Issues',
 'Cancellation - Bulky Trash',
 'Cemetery For-Profit Complaint',
 'Channel Maintenance',
 'Child & Adult Protective Services Adoption Foster Care',
 'Citation - Code Enforcement',
 'Civil Expungements',
 'Clean Lot',
 'Clean Water Act Fee',
 'Comprehensive Community Clean-up - CCCC',
 'Concrete Work (Contractor)',
 'Concrete Work (In House)',
 'Construction Issues'

#### Difference of service description between 2019 and 2020

In [11]:
sym_diff_service_desc = set(service_desc_2019) ^ set(service_desc_2020)
print(len(sym_diff_service_desc))
sym_diff_service_desc

38


{'Abandoned Construction',
 'Animal - Commercial Facility Complaint',
 'Animal - Pet License Application',
 'Animal - Pet Waste',
 'Animal - Volunteering',
 'Board Structure',
 'Building Maintenance (Internal)',
 'CEX - Correspondence',
 'Call-a-Bus Program Information',
 'Call-a-Cab',
 'Children Youth and Families',
 'Clean Lot/Board Structure',
 'Coronavirus/COVID-19',
 'County Council Record-Keeping',
 'CountyClick 311 Site Enhancement Suggestion',
 'Court Case - Code Enforcement',
 'Court Paper Status',
 'Expedited Food Supplement Benefit (Food Stamps)',
 'Food Truck(s) in a Hub - Food Complaints Only',
 'Food Truck(s) in a Hub - Other Complaints',
 'Mediation Request',
 'Open Enrollment',
 'PS - Theft',
 'PS - Vandalism',
 'Section 8 (Housing Choice Voucher) Concern',
 'Sign Blitz',
 'Status of Open Warrants',
 'Temporary Cash Assistance (TCA) Program',
 'Town Hall - COVID-19',
 'Under Drain - Remove and Replace',
 'Vehicle - Remove Vehicle Requested by Property Owner',
 'Vehicle 

1. Services only appeared 2019

In [12]:
diff_service_desc_2019 =  set(service_desc_2019) - set(service_desc_2020)
print(len(diff_service_desc_2019))
diff_service_desc_2019_ls = list(diff_service_desc_2019)

21


In [13]:
PGC_311_2019_df.loc[PGC_311_2019_df['Request Name'].isin(diff_service_desc_2019_ls)].groupby('Request Name').size()

Request Name
Abandoned Construction                                    1
Animal - Commercial Facility Complaint                    8
Animal - Pet Waste                                        2
Board Structure                                           9
Building Maintenance (Internal)                           1
Call-a-Bus Program Information                            6
Call-a-Cab                                                3
Children Youth and Families                               1
Clean Lot/Board Structure                                57
County Council Record-Keeping                             1
CountyClick 311 Site Enhancement Suggestion              12
Court Case - Code Enforcement                           351
Food Truck(s) in a Hub - Food Complaints Only             1
Food Truck(s) in a Hub - Other Complaints                 3
Mediation Request                                         1
Open Enrollment                                         575
Section 8 (Housing Choice V

2. Services only appeared in 2020

In [14]:
diff_service_desc_2020 =  set(service_desc_2020) - set(service_desc_2019) 
print(len(diff_service_desc_2020))
diff_service_desc_2020_ls = list(diff_service_desc_2020)

17


In [15]:
PGC_311_2020_df.loc[PGC_311_2020_df['Request Name'].isin(diff_service_desc_2020_ls)].groupby('Request Name').size()

Request Name
Animal - Pet License Application                      9
Animal - Volunteering                                 1
CEX - Correspondence                                  2
Coronavirus/COVID-19                                 35
Court Paper Status                                    1
Expedited Food Supplement Benefit (Food Stamps)       2
PS - Theft                                            1
PS - Vandalism                                        1
Status of Open Warrants                               1
Temporary Cash Assistance (TCA) Program               2
Town Hall - COVID-19                                 92
Under Drain - Remove and Replace                      2
Vehicle Audit - Patrol                                3
Violations - COVID-19                              1559
Work Order - Street & Drainage Maintenance            1
Zoning - Stats (Other)                                1
Zoning Periodic Inspections                           1
dtype: int64

In [16]:
PGC_311_2019_df['added_month'] = PGC_311_2019_df['opened_date'].dt.month
df = PGC_311_2019_df.groupby(['added_month','Request Name']).size().reset_index(name='count')
service_count_2019 = pd.DataFrame(columns = df['added_month'].unique().tolist(), index = df['Request Name'].unique().tolist())
for row in range(df.shape[0]):
    setvice_type = df.at[row, 'Request Name']
    month = df.at[row, 'added_month']
    service_count_2019.at[setvice_type, month] = df.at[row, 'count']
service_count_2019.fillna(0, inplace = True)
service_count_2019 = service_count_2019.add_prefix('2019_')
service_count_2019['total #reports_2019'] = service_count_2019.sum(axis=1)
service_count_2019.sort_values(by = 'total #reports_2019', ascending=False, inplace=True)
service_count_2019.reset_index(inplace=True)
service_count_2019.rename(columns = {'index': 'Request Name'}, inplace = True)
service_count_2019.head(10)


,Request Name,2019_1,2019_2,2019_3,2019_4,2019_5,2019_6,2019_7,2019_8,2019_9,2019_10,2019_11,2019_12,total #reports_2019
0,ENF-Residential Property Concerns,429,402,645,687,879,696,607,557,541,447,282,273,6445
1,Pothole Repair,529,844,917,584,473,368,296,175,133,130,158,204,4811
2,PSC Audio Recording Request,1,0,193,465,411,347,533,412,358,458,392,402,3972
3,Parking Complaints,274,241,389,294,315,303,322,312,294,281,243,218,3486
4,CountyClick 311 Emails,0,54,137,508,794,367,599,558,330,2,2,0,3351
5,Illegal Dumping Removal (from County Right-of-...,221,217,291,218,250,248,332,267,297,301,209,190,3041
6,Cancellation - Bulky Trash,156,127,150,127,155,187,203,182,164,185,122,145,1903
7,ENF-Rental Issues Apartment Unit,104,97,85,104,126,126,235,182,136,122,106,92,1515
8,Tree Trimming in Public Right-of-Way,36,36,75,83,188,231,256,200,114,102,70,33,1424
9,Vehicle - Abandoned (No Plates),135,118,148,144,129,120,90,91,108,99,71,78,1331


In [17]:
PGC_311_2020_df['added_month'] = PGC_311_2020_df['opened_date'].dt.month
df = PGC_311_2020_df.groupby(['added_month','Request Name']).size().reset_index(name='count')
service_count_2020 = pd.DataFrame(columns = df['added_month'].unique().tolist(), index = df['Request Name'].unique().tolist())
for row in range(df.shape[0]):
    setvice_type = df.at[row, 'Request Name']
    month = df.at[row, 'added_month']
    service_count_2020.at[setvice_type, month] = df.at[row, 'count']
service_count_2020.fillna(0, inplace = True)
service_count_2020 = service_count_2020.add_prefix('2020_')
service_count_2020['total #reports_2020'] = service_count_2020.sum(axis=1)
service_count_2020.sort_values(by = 'total #reports_2020', ascending=False, inplace=True)
service_count_2020.reset_index(inplace=True)
service_count_2020.rename(columns = {'index': 'Request Name'}, inplace = True)
service_count_2020.head(10)

,Request Name,2020_1,2020_2,2020_3,2020_4,2020_5,2020_6,2020_7,2020_8,2020_9,2020_10,2020_11,2020_12,total #reports_2020
0,PSC Audio Recording Request,427,446,529,462,388,476,433,404,518,492,537,491,5603
1,ENF-Residential Property Concerns,355,365,370,236,400,522,452,531,571,661,412,391,5266
2,Illegal Dumping Removal (from County Right-of-...,274,222,194,165,173,284,348,327,314,294,291,288,3174
3,Parking Complaints,272,235,169,133,151,221,267,293,340,311,281,221,2894
4,Pothole Repair,321,391,215,74,128,148,134,164,131,89,130,134,2059
5,Cancellation - Bulky Trash,138,113,99,0,0,0,42,252,240,280,243,185,1592
6,Violations - COVID-19,0,0,0,68,114,96,208,199,137,154,283,300,1559
7,Tree Trimming in Public Right-of-Way,49,56,35,70,113,175,240,208,173,146,76,32,1373
8,Storm Drain Maintenance,54,91,43,68,76,84,119,168,213,107,136,90,1249
9,ENF-Rental Issues Apartment Unit,91,86,59,59,75,105,157,163,120,107,120,103,1245


In [18]:
# remove the last column
merge_service_count = pd.merge(service_count_2019.iloc[:,0:-1], service_count_2020.iloc[:,0:-1], on = 'Request Name', how = 'outer')
merge_service_count.fillna(0, inplace = True)
merge_service_count.head()

,Request Name,2019_1,2019_2,2019_3,2019_4,2019_5,2019_6,2019_7,2019_8,2019_9,...,2020_3,2020_4,2020_5,2020_6,2020_7,2020_8,2020_9,2020_10,2020_11,2020_12
0,ENF-Residential Property Concerns,429.0,402.0,645.0,687.0,879.0,696.0,607.0,557.0,541.0,...,370.0,236.0,400.0,522.0,452.0,531.0,571.0,661.0,412.0,391.0
1,Pothole Repair,529.0,844.0,917.0,584.0,473.0,368.0,296.0,175.0,133.0,...,215.0,74.0,128.0,148.0,134.0,164.0,131.0,89.0,130.0,134.0
2,PSC Audio Recording Request,1.0,0.0,193.0,465.0,411.0,347.0,533.0,412.0,358.0,...,529.0,462.0,388.0,476.0,433.0,404.0,518.0,492.0,537.0,491.0
3,Parking Complaints,274.0,241.0,389.0,294.0,315.0,303.0,322.0,312.0,294.0,...,169.0,133.0,151.0,221.0,267.0,293.0,340.0,311.0,281.0,221.0
4,CountyClick 311 Emails,0.0,54.0,137.0,508.0,794.0,367.0,599.0,558.0,330.0,...,1.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,0.0


In [19]:
merge_service_count.columns

Index(['Request Name', '2019_1', '2019_2', '2019_3', '2019_4', '2019_5',
       '2019_6', '2019_7', '2019_8', '2019_9', '2019_10', '2019_11', '2019_12',
       '2020_1', '2020_2', '2020_3', '2020_4', '2020_5', '2020_6', '2020_7',
       '2020_8', '2020_9', '2020_10', '2020_11', '2020_12'],
      dtype='object')

In [20]:
pre_covid_cols = ['Request Name', '2019_1', '2019_2', '2019_3', '2019_4','2019_5', '2019_6', '2019_7', '2019_8', '2019_9', '2019_10', '2019_11','2019_12', '2020_1', '2020_2']
covid_cols = ['Request Name', '2020_3', '2020_4', '2020_5', '2020_6','2020_7', '2020_8', '2020_9', '2020_10', '2020_11','2020_12']
pre_covid_service_count = merge_service_count[pre_covid_cols]
covid_service_count = merge_service_count[covid_cols]

In [21]:
pre_covid_service_count['total #reports before lockdown'] = pre_covid_service_count.sum(axis=1)
covid_service_count['total #reports after lockdown'] = covid_service_count.sum(axis=1)

In [22]:
pre_covid_service_count.sort_values(by = 'total #reports before lockdown', ascending=False, inplace=True)
pre_covid_service_count.reset_index(drop= True, inplace=True)
covid_service_count.sort_values(by = 'total #reports after lockdown', ascending=False, inplace=True)
covid_service_count.reset_index(drop= True, inplace=True)

In [23]:
pre_covid_types = pre_covid_service_count['Request Name'].tolist()
covid_types = covid_service_count['Request Name'].tolist()

In [24]:
pre_covid_service_count
total_service_types = list(set(pre_covid_types + covid_types))
len(total_service_types)

194

### Perform t-test on #reports for each type across months
* To understand the difference before and after lock down (lock down: March - Dec. 2020)

In [25]:
t_test_df = pd.DataFrame(columns=['service_type','p-value','direction'])
for service_type in total_service_types:
    pre_covid_var = pre_covid_service_count.loc[pre_covid_service_count['Request Name']== service_type][pre_covid_cols[1:]].values.flatten()
    covid_var = covid_service_count.loc[covid_service_count['Request Name']== service_type][covid_cols[1:]].values.flatten()
    t_val, p_val = stats.ttest_ind(pre_covid_var, covid_var, equal_var = False)
    direction = covid_var.mean() - pre_covid_var.mean()
    t_test_res = {'service_type': service_type, 'p-value': p_val, 'direction': direction}
    t_test_df = t_test_df.append(t_test_res,ignore_index = True)

In [26]:
t_test_df.sort_values(by = 'p-value', inplace=True)
t_test_df.reset_index(drop=True, inplace = True)
t_test_df

,service_type,p-value,direction
0,Rental License Inspection (Single Family),0.000044,39.157143
1,Food Facility Complaint,0.000102,-21.542857
2,Court Case - Code Enforcement,0.000159,-25.071429
3,Food Supplement Program (Food Stamps),0.000422,1.900000
4,Violations - COVID-19,0.000525,155.900000
...,...,...,...
189,Fence Repair -Contractor,0.941048,0.028571
190,Handicapped Ramp - Repair,0.944872,-0.014286
191,zzz location test,0.954731,0.014286
192,Street Light New Install (PGC),0.961131,-0.071429


In [27]:
t_test_df.to_csv('../result/PGC_service_type_ttest.csv')

In [28]:
t_test_low_p = t_test_df.loc[t_test_df['p-value']<=0.05]
t_test_low_p.sort_values(by = ['direction','p-value'], inplace=True)
t_test_low_p.reset_index(drop = True, inplace = True)
t_test_low_p['inc/dec'] =['increased' if sign == True else 'decreased' for sign in (t_test_low_p['direction'] >0).tolist()]
inc_dec_df = t_test_low_p.groupby('inc/dec').size().reset_index()
inc_dec_df.rename(columns = {0:'#service types'}, inplace = True)
inc_dec_df

,inc/dec,#service types
0,decreased,32
1,increased,7


In [29]:
t_test_low_p.to_csv('../result/PGC_services_inc_dec.csv')
t_test_low_p

,service_type,p-value,direction,inc/dec
0,Pothole Repair,0.002042,-259.800000,decreased
1,CountyClick 311 Emails,0.007125,-238.800000,decreased
2,Vehicle - Abandoned (No Plates),0.013448,-26.285714,decreased
3,Court Case - Code Enforcement,0.000159,-25.071429,decreased
4,Food Facility Complaint,0.000102,-21.542857,decreased
5,Traffic Signs - Repair/Replace,0.005237,-19.885714,decreased
6,Vehicle Audit - Incoming Referral,0.009907,-19.371429,decreased
7,PSC Customer Service,0.010285,-19.314286,decreased
8,Road Resurfacing,0.011480,-15.128571,decreased
9,Cable Commission Complaint,0.003146,-15.028571,decreased


### Box plot for types that have p-value <0.05

In [30]:
low_p_types = t_test_low_p['service_type'].tolist()

In [31]:
pre_covid_box_plt = pre_covid_service_count.set_index('Request Name').T.iloc[:-1].reset_index()

covid_box_plt = covid_service_count.set_index('Request Name').T.iloc[:-1].reset_index()
covid_box_plt = covid_box_plt[pre_covid_box_plt.columns.to_list()]
box_plt_df = pre_covid_box_plt.append(covid_box_plt,ignore_index=True,sort=False)
box_plt_df.rename(columns={'index':'year_month'},inplace = True)
period_ls = ['before lockdown']* (pre_covid_box_plt.shape[0])
period_ls.extend(['after lockdown']* (covid_box_plt.shape[0]))
box_plt_df['period'] = period_ls
box_plt_df.head(10)

Request Name,year_month,ENF-Residential Property Concerns,Pothole Repair,PSC Audio Recording Request,Parking Complaints,Illegal Dumping Removal (from County Right-of-Way),CountyClick 311 Emails,Cancellation - Bulky Trash,ENF-Rental Issues Apartment Unit,Vehicle - Abandoned (No Plates),...,Animal - Pet License Application,Animal - Volunteering,Coronavirus/COVID-19,Town Hall - COVID-19,Violations - COVID-19,Court Paper Status,PS - Vandalism,PS - Theft,Work Order - Street & Drainage Maintenance,period
0,2019_1,429.0,529.0,1.0,274.0,221.0,0.0,156.0,104.0,135.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lockdown
1,2019_2,402.0,844.0,0.0,241.0,217.0,54.0,127.0,97.0,118.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lockdown
2,2019_3,645.0,917.0,193.0,389.0,291.0,137.0,150.0,85.0,148.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lockdown
3,2019_4,687.0,584.0,465.0,294.0,218.0,508.0,127.0,104.0,144.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lockdown
4,2019_5,879.0,473.0,411.0,315.0,250.0,794.0,155.0,126.0,129.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lockdown
5,2019_6,696.0,368.0,347.0,303.0,248.0,367.0,187.0,126.0,120.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lockdown
6,2019_7,607.0,296.0,533.0,322.0,332.0,599.0,203.0,235.0,90.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lockdown
7,2019_8,557.0,175.0,412.0,312.0,267.0,558.0,182.0,182.0,91.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lockdown
8,2019_9,541.0,133.0,358.0,294.0,297.0,330.0,164.0,136.0,108.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lockdown
9,2019_10,447.0,130.0,458.0,281.0,301.0,2.0,185.0,122.0,99.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before lockdown


In [33]:
num_imgs = math.ceil(len(low_p_types)/4)
sns.set_theme(style="whitegrid")
type_count =0
for img in range(num_imgs):
    if img < num_imgs -1:
        rows = 2
    else:
        rows = int(math.ceil( (len(low_p_types) - type_count)/2 ))
    fig, axes = plt.subplots(rows,2, figsize = (20,10))
    for row in range(rows):
        for col in range(2):
            if type_count < len(low_p_types):
                if rows >1:
                    sns.boxplot(x='period', y=low_p_types[type_count], data=box_plt_df, ax=axes[row, col])
                else:
                    sns.boxplot(x='period', y=low_p_types[type_count], data=box_plt_df, ax=axes[col])
                type_count +=1
    plt.savefig('../figure/PGC_ServiceTypesBoxPlt/box_plt_' + str(img +1))
    plt.close()